In [ ]:
import sqlite3
import pandas as pd
from tqdm import tqdm
import folium

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None) 


In [ ]:
fn = 'points.sqlite'
points = pd.read_sql('select * from points where not banned', sqlite3.connect(fn))

In [ ]:
points.head()

In [ ]:
points.info()

# activity

In [ ]:
df = points
df["datetime"] = df["datetime"].astype("datetime64[ns]")

plot = df['datetime'].groupby([df["datetime"].dt.year, df["datetime"].dt.month]).count().plot(kind="bar", title="Points per month", figsize=(50, 5))
# plot.figure.savefig("plots/points_per_month.png")

In [ ]:
df = points[['datetime', 'wait']].dropna()
df["datetime"] = df["datetime"].astype("datetime64[ns]")

plot = df['datetime'].groupby([df["datetime"].dt.year, df["datetime"].dt.month]).count().plot(kind="bar", title="Points per month", figsize=(50, 5))
# plot.figure.savefig("plots/points_per_month_with_waittime.png")

# history

In [ ]:
df = points[points["datetime"].isna()]
len(df)

In [ ]:
df.info()

In [ ]:
df = points.sort_values(by='datetime')
df.head(1)

In [ ]:
df[~df["nickname"].isna()].head(1)

In [ ]:
old = points[points["datetime"] < "2010-08-11"]
old = old[old["datetime"] > "2005-08-01"]
m = folium.Map(location=[old['lat'].mean(), df['lon'].mean()], zoom_start=2)
for _, row in tqdm(df.iterrows()):
    folium.CircleMarker(location=[row['lat'], row['lon']], radius=2).add_to(m)
m

In [ ]:
df[(df["datetime"] > "2017-01-01") & (df["datetime"] < "2020-05-01")]["nickname"].value_counts()
# no-one addinf spots consistntly in this period

In [ ]:
df[df["nickname"] == "Sitko"].tail()

In [ ]:
df[df["nickname"] == "Thumb-up"].tail()

In [ ]:
df[(df["datetime"] < "2017-01-01") & (df["datetime"] > "2005-05-01")]["nickname"].value_counts()


# top spots

In [ ]:
groups = points.groupby(["lat", "lon"])
places = groups[["country"]].first()
places["rating"] = groups.rating.mean()
places["points"] = groups.size()

In [ ]:
places['score'] = places['rating']**3 * places['points']
places.sort_values("score", ascending=False).head(5)

# worst spots

In [ ]:
points['rating_reversed'] = 6 - points['rating']
groups = points.groupby(["lat", "lon"])
places = groups[["country"]].first()
places["rating"] = groups.rating.mean()
places['rating_reversed'] = groups.rating_reversed.mean()
places["points"] = groups.size()

In [ ]:
places['score'] = places['rating_reversed']**3 * places['points']
places.sort_values("score", ascending=False).head(20)

# top contributors

In [ ]:
points.name.value_counts().head(20)

In [ ]:
points[points.datetime.dt.year == 2024].name.value_counts().head(20)